In [1]:
import ollama

In [2]:
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

The sky appears blue to our eyes because of a phenomenon called scattering, which is the way that light interacts with tiny particles in the Earth's atmosphere.

Here's what happens:

1. **Sunlight**: The sun emits white light, which contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Atmosphere**: When this sunlight enters our atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.
3. **Scattering**: As the light interacts with these molecules, it gets scattered in all directions. This is because the molecules are much larger than the wavelength of light, so they can't absorb or reflect the light efficiently. Instead, they scatter the shorter wavelengths of light (like blue and violet) more than the longer wavelengths (like red and orange).
4. **Blue dominance**: As a result of this scattering, the blue and violet parts of the sunlight a

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = "llama3"

In [4]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from operator import itemgetter

# TODO: Using Memory search for now. Good idea to switch to vector database in future
from langchain_community.vectorstores import DocArrayInMemorySearch

model = Ollama(model=MODEL)
parser = StrOutputParser()

In [5]:
# Input template
template = """Answer the question based on the context below. If you can't answer the question, reply "I don't know".
Context: {context}

Question: {question}"""


In [6]:
prompt = PromptTemplate.from_template(template = template)
prompt.format(context="Here is some context", question="Here is a question")


'Answer the question based on the context below. If you can\'t answer the question, reply "I don\'t know".\nContext: Here is some context\n\nQuestion: Here is a question'

# For ben, please input the code to read out the PDFs / other documents here. If more refactoring is needed, we can just do for PDF first, and work on other file types later on.

In [7]:
# To read PDFs and create the embeddings
# sample code for editing  - Ben
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
# "C:\Users\TechD\Documents\Marcus\LLMs\Rarediseases2024\data\HPP Papers (PDF) Via Liezl Puzon\10.1002___ajmg.a.33146.pdf"
loader = PyPDFLoader(os.path.join("data", "HPP Papers (PDF) Via Liezl Puzon", "10.1002___ajmg.a.33146.pdf"))
pages = loader.load_and_split()
embeddings = OllamaEmbeddings(model=MODEL)

In [8]:
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [9]:
retriever = vectorstore.as_retriever()

In [10]:
chain = (
    {
        "context": itemgetter("question") | retriever, 
        "question":itemgetter("question") 

    }
    | prompt
    | model
    | parser
)

In [11]:
questions = [
    # Input questions
    "What is HPP?",
    "Can you give me a summary of HPP?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question':question})}")

Question: What is HPP?
Answer: Based on the context, I don't know what HPP stands for, but it appears to be related to a medical condition or disorder, possibly Cleidocranial dysplasia, given the presence of radiographic images and genetic information.
Question: Can you give me a summary of HPP?
Answer: Based on the provided context, Hypophosphatasia (HPP) is described as a distinct skeletal disorder characterized by severe osteopenia, low alkaline phosphatase, and features such as Bowdler spurs. The text does not provide a detailed summary of HPP, but it mentions that the patient in question has clinical features which mimic hypophosphatasia.
